In [1]:
import os
import yaml
import json
import pandas as pd

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [4]:
import wombat.bsub as bsub

make test input yaml

In [6]:
template = yaml.safe_load(open('../tests/data/omero_wrapper/inputs_upload.yaml'))
template

{'dataset': 'HT308B1-S1H5A4_U1_20230526',
 'filepath': {'class': 'File',
  'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/sandbox/test_images/HT308B1-S1H5A4_right_05262023.ome.tiff'},
 'group': 'HTAN',
 'image_name': 'HT308B1-S1H5A4_U1_20230526',
 'project': 'Multiplex_Imaging'}

In [12]:
template = {
    'sample': 'HT308B1-S1H5A4_U1_test_20230526',
    'input_tif': {
        'class': 'File',
        'path': '/storage1/fs1/dinglab/Active/Projects/estorrs/sandbox/test_images/HT308B1-S1H5A4_right_05262023.ome.tiff'
    },
    'bbox': '5000,7000,5000,7000',
    'platform': 'phenocycler',
    'group': 'HTAN',
    'project': 'Multiplex_Imaging'
}

In [13]:
yaml.safe_dump(template, open('../tests/data/image_preprocessing_workflow/inputs.yaml', 'w'))

make cromwell compute1 config

In [16]:
workflow_root = '/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/image_preprocessing_workflow'
output_fp = '../tests/data/image_preprocessing_workflow/image_preprocessing_workflow.cromwell-config-db.compute1.dat'
bsub.save_compute1_cromwell_template(workflow_root, output_fp)

In [17]:
# !mkdir -p ../tests/data/image_preprocessing_workflow

make execution script

In [18]:
TEST_DIR_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/image_preprocessing_workflow'
TOOL_ROOT = '/storage1/fs1/dinglab/Active/Projects/estorrs/multiplex-imaging-pipeline'

dconfig = os.path.join(TEST_DIR_ROOT, 'image_preprocessing_workflow.cromwell-config-db.compute1.dat')
cwl_fp = os.path.join(TOOL_ROOT, 'cwl', 'image_preprocessing_workflow.cwl')
inputs_fp = os.path.join(TEST_DIR_ROOT, 'inputs.yaml')
volumes = [TEST_DIR_ROOT, TOOL_ROOT, workflow_root, '/storage1/fs1/dinglab', '/scratch1/fs1/dinglab']


In [19]:
args = bsub.DEFAULT_ARGS
args['group_name'] = 'test_cromwell'
args

{'mem': 10,
 'n_processes': 1,
 'max_mem': None,
 'docker': 'python:3.8',
 'queue': 'dinglab',
 'gpu_model': 'TeslaV100_SXM2_32GB',
 'gpu_mem': '30',
 'gpu_num': 1,
 'use_gpu': False,
 'group': 'compute-dinglab',
 'group_name': 'test_cromwell',
 'n_concurrent': 10,
 'interactive': False,
 'username': 'estorrs'}

In [30]:
run_commands = bsub.cromwell_commands(dconfig, cwl_fp, inputs_fp, args, volumes, workflow_root=workflow_root, log_fp='log.txt')
filepath = os.path.join('../tests/data/omero_wrapper/run_job_upload.sh')
bsub.write_command_file(run_commands, filepath)

In [31]:
for c in run_commands: print(c)

mkdir -p /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/omero_wrapper/upload/cromwell-workdir/logs
source /opt/ibm/lsfsuite/lsf/conf/lsf.conf
export LSF_DOCKER_NETWORK=host
export LSF_DOCKER_VOLUMES="/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/omero_wrapper:/storage1/fs1/dinglab/Active/Projects/estorrs/wombat/tests/data/omero_wrapper /storage1/fs1/dinglab/Active/Projects/estorrs/omero-wrapper:/storage1/fs1/dinglab/Active/Projects/estorrs/omero-wrapper /scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/omero_wrapper/upload:/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/omero_wrapper/upload /storage1/fs1/dinglab:/storage1/fs1/dinglab /scratch1/fs1/dinglab:/scratch1/fs1/dinglab"
bgadd -L 10 /estorrs/test_cromwell
export PATH="/opt/java/openjdk/bin:$PATH"
bsub -R 'select[mem>10GB] rusage[mem=10GB] span[hosts=1]' -M 11GB -n 1 -q dinglab -G compute-dinglab -a 'docker(estorrs/cromwell-runner:58)' -g /estorrs/test_cromwell -J db6de13a-eae8-4b89-

In [32]:
workflow_root = '/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/omero_wrapper/download'
output_fp = '../tests/data/omero_wrapper/download.cromwell-config-db.compute1.dat'
bsub.save_compute1_cromwell_template(workflow_root, output_fp)

dconfig = os.path.join(TEST_DIR_ROOT, 'download.cromwell-config-db.compute1.dat')
cwl_fp = os.path.join(TOOL_ROOT, 'cwl', 'omero_wrapper_download.cwl')
inputs_fp = os.path.join(TEST_DIR_ROOT, 'inputs_download.yaml')
volumes = [TEST_DIR_ROOT, TOOL_ROOT, workflow_root, '/storage1/fs1/dinglab', '/scratch1/fs1/dinglab']

run_commands = bsub.cromwell_commands(dconfig, cwl_fp, inputs_fp, args, volumes, workflow_root=workflow_root, log_fp='log.txt')
filepath = os.path.join('../tests/data/omero_wrapper/run_job_download.sh')
bsub.write_command_file(run_commands, filepath)

In [33]:
workflow_root = '/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/omero_wrapper/get_thresholds'
output_fp = '../tests/data/omero_wrapper/get_thresholds.cromwell-config-db.compute1.dat'
bsub.save_compute1_cromwell_template(workflow_root, output_fp)

dconfig = os.path.join(TEST_DIR_ROOT, 'get_thresholds.cromwell-config-db.compute1.dat')
cwl_fp = os.path.join(TOOL_ROOT, 'cwl', 'omero_wrapper_get_thresholds.cwl')
inputs_fp = os.path.join(TEST_DIR_ROOT, 'inputs_get_thresholds.yaml')
volumes = [TEST_DIR_ROOT, TOOL_ROOT, workflow_root, '/storage1/fs1/dinglab', '/scratch1/fs1/dinglab']

run_commands = bsub.cromwell_commands(dconfig, cwl_fp, inputs_fp, args, volumes, workflow_root=workflow_root, log_fp='log.txt')
filepath = os.path.join('../tests/data/omero_wrapper/run_job_get_thresholds.sh')
bsub.write_command_file(run_commands, filepath)

In [34]:
workflow_root = '/scratch1/fs1/dinglab/estorrs/cromwell-data/pecgs/testing/omero_wrapper/test'
output_fp = '../tests/data/omero_wrapper/test.cromwell-config-db.compute1.dat'
bsub.save_compute1_cromwell_template(workflow_root, output_fp)

dconfig = os.path.join(TEST_DIR_ROOT, 'test.cromwell-config-db.compute1.dat')
cwl_fp = os.path.join(TOOL_ROOT, 'cwl', 'omero_wrapper_test.cwl')
inputs_fp = os.path.join(TEST_DIR_ROOT, 'inputs_test.yaml')
volumes = [TEST_DIR_ROOT, TOOL_ROOT, workflow_root, '/storage1/fs1/dinglab', '/scratch1/fs1/dinglab']

run_commands = bsub.cromwell_commands(dconfig, cwl_fp, inputs_fp, args, volumes, workflow_root=workflow_root, log_fp='log.txt')
filepath = os.path.join('../tests/data/omero_wrapper/run_job_test.sh')
bsub.write_command_file(run_commands, filepath)